# 割引状態訪問頻度

---

今回は割引状態訪問頻度について解説していきたいと思います．

まず，割引状態訪問頻度の定義から確認していきます．

$$
{\rho}_\pi(s, a) \stackrel{\text { def }}{=} \sum_{t=0}^{\infty} \gamma^t P\left(s_t=s, a_t=a \mid \mu_0, \pi, \mathcal{T}\right)
$$

この式の意味は，方策$\pi$でどれだけ(s,a)が訪問されたかについての確率です．

例えば，この記法は次のような時に使用されます．

無限ホライゾンでの割引報酬和$\eta(\pi)$について次のように考えます．

$$
\eta(\pi)=\mathbb{E}_{s_0, a_0, \ldots}\left[\sum_{t=0}^{\infty} \gamma^t r\left(s_t\right)\right]
$$

これを展開すると，本来はこのような式になります．

$$
\eta(\pi)=\sum_s \sum_{t=0}^{\infty} \gamma^t P\left(s_t=s \mid {\pi}\right) \sum_a {\pi}(a \mid s) r(s,a) \\
 =\sum_s \sum_a \rho_{{\pi}}(s,a) r(s,a)

$$


と書けます．便利ですね，

---

この状態訪問頻度を動的計画法に適用する際のコードを書いていきます．

今回は有限ホライゾンを使うので，割引係数$\gamma$は考えません．

In [19]:
import jax.numpy as jnp
import jax
import numpy as np
from typing import NamedTuple, Optional
from jax.random import PRNGKey

key = PRNGKey(0)

S = 10  # 状態集合のサイズ
A = 3  # 行動集合のサイズ
S_set = jnp.arange(S)  # 状態集合
A_set = jnp.arange(A)  # 行動集合
H = 5  # ホライゾン

# 報酬行列を適当に作ります
key, _ = jax.random.split(key)
rew = jax.random.uniform(key=key, shape=(H, S, A))
assert rew.shape == (H, S, A)


# 遷移確率行列を適当に作ります
key, _ = jax.random.split(key)
P = jax.random.uniform(key=key, shape=(H, S*A, S))
P = P / jnp.sum(P, axis=-1, keepdims=True)  # 正規化して確率にします
P = P.reshape(H, S, A, S)
np.testing.assert_allclose(P.sum(axis=-1), 1, atol=1e-6)  # ちゃんと確率行列になっているか確認します


# 初期状態分布を適当に作ります
key, _ = jax.random.split(key)
mu = jax.random.uniform(key, shape=(S,))
mu = mu / jnp.sum(mu)
np.testing.assert_allclose(mu.sum(axis=-1), 1, atol=1e-6)  # ちゃんと確率行列になっているか確認します


# 状態集合, 行動集合, 割引率, 報酬行列, 遷移確率行列が準備できたのでMDPのクラスを作ります

class MDP(NamedTuple):
    S_set: jnp.array  # 状態集合
    A_set: jnp.array  # 行動集合
    H: int  # ホライゾン
    rew: jnp.array  # 報酬行列
    P: jnp.array  # 遷移確率行列
    mu: jnp.array  # 初期分布
    optimal_Q: Optional[jnp.ndarray] = None  # 最適Q値

    @property
    def S(self) -> int:  # 状態空間のサイズ
        return len(self.S_set)

    @property
    def A(self) -> int:  # 行動空間のサイズ
        return len(self.A_set)


mdp = MDP(S_set, A_set, H, rew, P, mu)

print("状態数：", mdp.S)
print("行動数：", mdp.A)
print("ホライゾン：", mdp.H)

状態数： 10
行動数： 3
ホライゾン： 5


In [28]:
import jax
from functools import partial
import jax.numpy as jnp

@jax.jit
def compute_greedy_policy(Q: jnp.ndarray) -> jnp.ndarray:
    '''
    Q: (Horizon,State,Action)
    greedy_policy: (Horizon,State,Action)
    '''
    greedy_policy: jnp.ndarray = jnp.zeros_like(Q)
    H, S, A = Q.shape

    def body_fn(i: int, greedy_policy: jnp.ndarray) -> jnp.ndarray:
        greedy_policy = greedy_policy.at[i, jnp.arange(S), Q[i].argmax(axis=-1)].set(1)
        return greedy_policy
    
    greedy_policy = jax.lax.fori_loop(0,H,body_fn,greedy_policy)

    return greedy_policy


def _compute_optimal_Q(mdp:MDP,H:int,S:int,A:int)->jnp.ndarray:
    # Initialize the optimal Q function with zeros
    optimal_Q = jnp.zeros((H+1,S,A))

    def backup(i,optimal_Q):
        # Compute the max Q over actions (S,A) -> (S,)
        max_Q = optimal_Q[i+1].max(axis=-1)
        # Update the Q function
        optimal_Q = optimal_Q.at[i].set(mdp.rew[i] + mdp.P[i] @ max_Q)
        return optimal_Q
    
    # Apply the backup to all times
    optimal_Q = jax.lax.fori_loop(0,H+1,backup,optimal_Q)
    return optimal_Q[:-1]

compute_optimal_Q = lambda mdp: _compute_optimal_Q(mdp,mdp.H,mdp.S,mdp.A)

@jax.jit
def compute_policy_Q(mdp:MDP,policy:jnp.ndarray) -> jnp.ndarray:
    H,S,A = policy.shape

    def backup(i,policy_Q):
        policy_V = mdp.P[i] @ (policy[i+1] * policy_Q[i+1]).sum(axis=-1)
        policy_Q = policy_Q.at[i].set(mdp.rew[i] + policy_V)
        return policy_Q
    
    policy_Q = jnp.zeros((H+1,S,A))
    policy_Q = jax.lax.fori_loop(0,H,backup,policy_Q)
    return policy_Q[:-1]

#Policy_matrix　(H,S,SA) each horizon，state about policy
@jax.jit
def policy_matrix(policy:jnp.ndarray) -> jnp.ndarray:
    H,S,A = policy.shape
    policy_matrix = jnp.zeros((H,S,S*A))
    for h in range(H):
        for s in range(S):
            policy_matrix = policy_matrix.at[h,s,s*A:(s+1)*A].set(policy[h,s])

    return policy_matrix

#Visit_occupancy_measure
@jax.jit
def compute_policy_visit(mdp:MDP,policy:jnp.ndarray,init_dist:jnp.ndarray) -> jnp.ndarray:
    H,S,A = policy.shape
    _policy_matrix = policy_matrix(policy)
    P = mdp.P.reshape(H,S*A,S)

    def body_fn(i,visit):
        next_visit = visit[i] @ P[i] @ _policy_matrix[i+1]
        visit = visit.at[i+1].set(next_visit)

        return visit
    
    visit = jnp.zeros((H+1,S*A))
    visit = visit.at[0].set(init_dist @ _policy_matrix[0])
    visit = jax.lax.fori_loop(0,H,body_fn,visit)
    visit = visit[:-1].reshape(H,S,A)

    return visit




In [39]:
optimal_Q = compute_optimal_Q(mdp)
optimal_V = optimal_Q.max(axis=-1)
optimal_policy = compute_greedy_policy(optimal_Q)
optimal_policy_Q = compute_policy_Q(mdp,optimal_policy)
mdp = mdp._replace(optimal_Q=optimal_Q)
print(f'最適Q値と最適方策を使ったQ値の差:{jnp.abs(optimal_Q - optimal_policy_Q).max():.3f}')

occuapancy_measure = compute_policy_visit(mdp,optimal_policy,mu)

for h in range(H):
    occ_policy = (occuapancy_measure * mdp.rew)[h:].sum(axis=0)
    return_DP = (optimal_Q[h] * occ_policy[h]).sum()
    # print(return_DP.shape,occ_policy.shape)
    print(f'訪問頻度によるDPとリターンの差:{np.abs(occ_policy - return_DP).max():.3f}')

最適Q値と最適方策を使ったQ値の差:0.000
訪問頻度によるDPとリターンの差:1.262
訪問頻度によるDPとリターンの差:1.468
訪問頻度によるDPとリターンの差:0.994
訪問頻度によるDPとリターンの差:0.797
訪問頻度によるDPとリターンの差:0.455


In [1]:
import jax

ModuleNotFoundError: jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.